<a href="https://colab.research.google.com/github/ajayjai30/SUBJECT_EXPERT_CHATBOT_FOR_STUDETNS_AND_RESEARCHERS/blob/main/PROJECT_SUBJECT_EXPERT_CHATBOT_PHASE_IV_FINAL_TESTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Section-1 Libraries and Dependencies Installation and setting up of libraries

Dependencies installation section

In [77]:
#dependencies installation
!pip install -q --upgrade google-generativeai langchain-google-genai
!sudo apt-get -y -qq install poppler-utils
!pip install langchain
!pip install langchain-community
!pip install gradio
!pip install pdf2image
!pip install -U langchain-chroma

Libraries Setting

In [78]:
#importing libraries
import os#for file handling purposes
from google.colab import userdata#used for storing public data when processing Here api will be stone and also for accesing secret access key.
import google.generativeai as genai#library for using LLM model with all features given by google
import urllib
from pathlib import Path as p
from pprint import pprint
import textwrap#For adjusting line breaks and adding indentation inside markdown.
from pdf2image import convert_from_path
from PIL import Image
import gradio as g
#import chromadb
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from IPython.display import display,Markdown#for displaying information as output
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA

***Function for editing the response style and also using text to speech methods for additional functionality***


Overall Functionality:

The to_markdown function takes plain text as input, converts bullet points to Markdown-style lists, indents all lines to create a blockquote, and returns the result as a Markdown object. This is useful for converting text into a format suitable for display on platforms that support Markdown, such as GitHub, Reddit, or various blogging platforms.

In [79]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#API key setup and verification of models availability

Section-1
setting up the configuration of Model from api key

In [80]:
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

verification of models for synchronizing with the environment

In [81]:
for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    print(model.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


#Environment creation and Initialization of model

Setting up the environment for the model

In [82]:
import os
os.environ['GOOGLE_API_KEY']=userdata.get("GOOGLE_API_KEY")

Initiating the model inside the environment

In [83]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

#Extracting text from pdf files located in drive

**Step-1:
Converting any document to pdf and splitting pdf pages to images and saving it in the local storage**

In [ ]:
def convert_document_to_single_page_images(document_path, output_dir):
    """Converts a document to individual one-page images and saves them in the specified directory.

    Args:
        document_path: The path to the document file.
        output_dir: The directory where the images will be saved.
    """

    extension = os.path.splitext(document_path)[1]

    if extension in [".pdf", ".docx"]:
        if extension == ".pdf":
            images = convert_from_path(document_path,strict=True,fmt='jpeg')
        #elif extension == ".docx":
            #text = docx2txt.process(document_path)
            # Assuming you have a function to convert text to images
            #images = convert_text_to_images(text, output_dir)

        # Save each image as a separate file
        for i, image in enumerate(images):
            image.save(f"{output_dir}/page_{i+1}.jpg")
    else:
        # Handle other document types (e.g., images, spreadsheets)
        # You might need additional libraries or custom logic
        raise NotImplementedError(f"Unsupported file format: {extension}")

document_path = input("enter input directory\n /content/drive/MyDrive/ContentforModelFluidMechanics/2500solvedproblemsinfluidmechanicshydraulics (1).pdf")
output_dir = input("enter output directory\n /content/drive/MyDrive/folder_for_converted_images_2")
convert_document_to_single_page_images(document_path, output_dir)

**Step-2: Extracting the text from the images using llm as a text extractor**

In [ ]:
def load_images_from_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.jpeg')]
    image_urls = [f"/{folder_path}/{file}" for file in image_files]
    return image_urls

# Replace 'path/to/your/folder' with the actual path to your image folder
folder_path = '/content/drive/MyDrive/folder_converted_images'
image_urls = load_images_from_folder(folder_path)

model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=userdata.get("GOOGLE_API_KEY"))


batch_size = 10
batches = [image_urls[i:i+batch_size] for i in range(0, len(image_urls), batch_size)]

# Initializing a list to store the responses
responses = []
count=0
for batch in batches:
    message = HumanMessage(
        content=[
            {
                "type": "text",
                "text": """extract all the text from the page,if it contains diagrams related to the questions,give an ample explanation about the image,
                also be careful on,the symbols that are comming throught the page, try to capture that as same as like that in the page,
                also i have given u an reference for the symbols, use them for your reference.
                Do not give the symbols as a single tabular representation i have just given it for your reference,
                Just use it as your reference,dont give any symbol reference"""
            },
            *[{
                "type": "image_url",
                "image_url": url
            } for url in batch]
        ]
    )

    # Sending the message to the model and appending the response to the list
    response = model.invoke([message])
    count+=1
    responses.append(response.content)
    print(count,'\n\n\n\n\n',response.content)

**Step-3: Saving the extracted text as a txt file in local storage**

In [ ]:
def write_list_to_file(list_of_text, file_path, folder_path):
    try:
        # Create the folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)

        # Construct the full file path
        full_file_path = os.path.join(folder_path, file_path)

        with open(full_file_path, 'w') as file:
            for item in list_of_text:
                file.write(item + '\n')
        print("List written to file successfully.")
    except IOError:
        print("Error: Could not write to file.")

file_name = input("Enter the file directory: \n extracted_text-1(from2500solved).txt")
folder_name = input("Enter the file directory: \n /content/drive/MyDrive/Extracted_text")

write_list_to_file(responses, file_name, folder_name)

#Building pipeline for rag

Here the flow of pipeline is shown below


**pdf(or any kind of document) ==> converted into separate images text extracted using llm ==> text generated as chunks ==> Chunks converted into embeddings ==> uploaded into chroma data base**

In [84]:
# prompt: open the text file and save it in all_pages variable

#opening the text file and saving it in all_pages variable
with open('/content/drive/MyDrive/Extracted_text/extracted_text-1(from2500solved).txt', 'r') as file:
    all_pages = file.read()


In [85]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=10)
text_chunks=text_splitter.split_text(all_pages)

In [86]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=userdata.get("GOOGLE_API_KEY"))

In [87]:
vector_index = Chroma.from_texts(text_chunks, embeddings).as_retriever(search_kwargs={"k":5})

#Question answer bot final step before fine tuning the answers

In [97]:
template_for_prompt_response="""
Use the following pieces of context to answer the question at the end. If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.
dont make up your own answers. Explain to the question with maximum answer that u can give. Also try to solve the problemson your own and understand the contents and generate answers on your own. Dont tell that this
is the content i found for the question.Also dont mention like this 'The question you've asked is related to the context you provided.'. Do it like you are solving the problem
and explaining the students and researchers,also dont forget that you need to solve the sums and should give answers.

{context}
Question: {question}
Helpful Answer:
"""
Question_answer_chain_promt_adjusting_control=PromptTemplate.from_template(template_for_prompt_response)

In [98]:
Question_answer_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={'prompt':Question_answer_chain_promt_adjusting_control}
)

#Image to prompt generator

In [90]:
def image_to_prompt(image=None):
  if image is None:
    return "No image provided"
  else:
    model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=userdata.get("GOOGLE_API_KEY"))
    prompt = """Extract the text from the image and summarize the content concisely. Provide a clear and informative summary."""
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": image}
        ]
    )

    try:
        response = model.invoke([message])
        print(response.content)
        return response.content
    except Exception as e:
        print("Error processing image:", e)
        return "Error on processing Image"

#Testing with temporary hosting

In [101]:
def text_response(Type_your_Question_here,Drop_Your_Image_Here=None):
   imageresult=image_to_prompt(Drop_Your_Image_Here)
   result=Question_answer_chain({"query":Type_your_Question_here+imageresult})
   return result['result']
# Create a Gradio interface
interface = g.Interface(fn=text_response, inputs=['text',g.Image(type='filepath')],
                        outputs='markdown',
                        description='This chatbot is trained on a particular subject with the contents available related to the particular subject.',
                        title='SUBJECT EXPERT CHATBOT PHASE-IV FINAL TEST')

In [102]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bb1deb4c3adcb34657.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
